### 샘플 데이터
```
{'_id': '61ffbb81f6b8e9bd026bb500',
 'data': {'attachments': {},
  'author_id': '1294668508886949892',
  'conversation_id': '1490297649379037186',
  'created_at': '2022-02-06T12:13:42.000Z',
  'entities': {'mentions': [{'start': 3,
     'end': 13,
     'username': 'IconB2000',
     'id': '1317575082991099906'}]},
  'geo': {},
  'id': '1490297649379037186',
  'lang': 'en',
  'non_public_metrics': {},
  'organic_metrics': {},
  'possibly_sensitive': 'false',
  'promoted_metrics': {},
  'public_metrics': {'retweet_count': 2256,
   'reply_count': 0,
   'like_count': 0,
   'quote_count': 0},
  'referenced_tweets': [{'type': 'retweeted', 'id': '1490022206759485448'}],
  'reply_settings': 'everyone',
  'source': 'Twitter for Android',
  'text': 'RT @IconB2000: No food for lazy man but lazy woman dey chop steady steady\nThis story no clear'},
 '_timestamp': '2022-02-06T21:13:53.035409+09:00'}
```

```
# mongodb 
데이터베이스 > 컬렉션 > 도큐먼트
# 데이터베이스 선택
show databases
show dbs
use twitter

# 데이터베이스에서 컬렉션 조회
show collections

# 도큐먼트 조회 및 count 
db.tweet_sample.find({})
db.tweet_sample.find({}).limit(1)

db.tweet_sample.find({"data.lang": "ko"})
db.tweet_sample.find({"data.lang": "ko"}).limit(1)
db.tweet_sample.find({"data.lang": "ko"}).count()

db.tweet_sample.find({"_id" : ObjectId("61ffbb80f6b8e9bd026bb4f6")})
db.tweet_sample.find({"_timestamp" : "2022-02-06T21:13:52.836027+09:00"}).pretty()
db.tweet_sample.find({"_timestamp" : {$gt: "2022-02-06T21:13:52.836027+09:00"}})
db.tweet_sample.find({"_timestamp" : {$gt: "2022-02-06T21:13:52.836027+09:00"}}).count()
```

```
# Distinct 및 Group by
# distint: https://www.geeksforgeeks.org/mongodb-distinct-method/
# group by: https://www.fun-coding.org/mongodb_advanced4.html
# group by: https://docs.mongodb.com/manual/reference/operator/aggregation/group/
db.tweet_sample.distinct("data.lang")
db.tweet_sample.aggregate( [ 
	// group by 
	{ $group : {
		_id : "$data.lang",
		count: { $sum: 1}  
		}
	},
	// 정렬
	{ $sort: {count: -1}
	}
] )
```

```
mongodb 데이터 복사하기
참고자료: https://stackoverflow.com/questions/11554762/how-to-copy-a-collection-from-one-database-to-another-in-mongodb
db.tweet_sample.find().limit(1000000).forEach(function(p) {db.tweet_small_sample.insert(p);} );
```

### MongoDB 검색 참고자료
- 참고자료1: https://pro-self-studier.tistory.com/59
- 참고자료2: https://rastalion.me/mongodb-%EB%8F%84%ED%81%90%EB%A8%BC%ED%8A%B8-%EC%A1%B0%ED%9A%8C%ED%95%98%EA%B8%B0/

### 예시
```
db.tweet_sample.find({"data.lang": "en"}
```

In [2]:
import pandas as pd
import pymongo

mongo = pymongo.MongoClient()

In [3]:
def tweets(*args, **kwargs):
    for tweet in mongo.twitter.tweet_sample.find(*args, **kwargs):
        if "delete" not in tweet:
            yield {
                "id": tweet["_id"],
                "data": tweet["data"],
                "text": tweet["data"]["text"],
                # "geo" : tweet["geo"],
                "process_time" : tweet["_timestamp"],
            }   

In [6]:
pd.DataFrame(tweets({"data.lang": 'ko'}, limit=100000))

,id,data,text,process_time
0,61ffbb80f6b8e9bd026bb4f6,"{'attachments': {}, 'author_id': '113664549401...",@dkdtmxkcoth 나도 같커잖아...,2022-02-06T21:13:52.836027+09:00
1,61ffbb81f6b8e9bd026bb4ff,"{'attachments': {}, 'author_id': '131857477421...",RT @Rmlove09127: 아크릴 공구\n#RT 추첨 1분께 1세트 보내드릴게요...,2022-02-06T21:13:53.025851+09:00
2,61ffbb81f6b8e9bd026bb503,"{'attachments': {}, 'author_id': '132093495330...",RT @BYangsalang: 임영웅 [따라따라] 사콜ver. 7백만뷰돌파 🎊\n아...,2022-02-06T21:13:53.215722+09:00
3,61ffbb81f6b8e9bd026bb509,"{'attachments': {}, 'author_id': '2404429310',...","모두들 힘껏 뛰었지만 늦어 버렸어. 자넬리도 말이야, 열심히 뛰었는데 쫓아오지 못했어.",2022-02-06T21:13:53.434050+09:00
4,61ffbb81f6b8e9bd026bb51d,"{'attachments': {}, 'author_id': '3042236256',...",RT @Chuverall: 이게 덕질할 때 2년 법칙이 있는데\n뭔가 열심히 덕질한...,2022-02-06T21:13:53.523936+09:00
...,...,...,...,...
99995,6200c4dcf6b8e9bd029b0a4b,"{'attachments': {}, 'author_id': '131161043394...",@sio_r_2657 (๑•̀ㅁ•́ฅ✧,2022-02-07T16:06:04.962235+09:00
99996,6200c4dcf6b8e9bd029b0a4c,{'attachments': {'media_keys': ['3_14905822732...,KLAY APE CLUB이 신규 아BABY APE 캐릭터 론칭을 기념해 빅이벤트를 ...,2022-02-07T16:06:04.989730+09:00
99997,6200c4ddf6b8e9bd029b0a63,"{'attachments': {}, 'author_id': '2923053194',...",야 영길아 이 썩을놈아 여비서 한테 약사오라고 \n했냐? 무슨약 전립선 약 \n남...,2022-02-07T16:06:05.475041+09:00
99998,6200c4ddf6b8e9bd029b0a66,{'attachments': {'media_keys': ['3_14902692494...,RT @MBEST00M: 근데 이거 대니곰인데 https://t.co/huzcpEs10S,2022-02-07T16:06:05.490972+09:00


In [ ]:
mongo.twitter.tweet_sample.distinct("data.lang")

```
# 해시태그가 있는 경우만 조회
# 참고자료: http://daplus.net/mongodb-mongodb%EC%97%90-%ED%95%84%EB%93%9C%EA%B0%80-%EC%9E%88%EB%8A%94%EC%A7%80-%ED%99%95%EC%9D%B8/
# API상세정보링크 : https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/tweet

db.tweet_sample.find({"data.entities.hashtags": {"$exists":true}})

# 해시태그가 있으면서 한국어만 조회: mongodb And 조건
# 참고자료: https://velopert.com/479
db.tweet_sample.find({ $and: [ { "data.lang": "ko" }, { "data.entities.hashtags": { "$exists":true }} ] })
db.tweet_sample.find({ $and: [ { "data.lang": "ko" }, { "data.entities.hashtags": { "$exists":true }} ] }).limit(5).pretty()
{
        "_id" : ObjectId("61ffbb88f6b8e9bd026bb692"),
        "data" : {
                "attachments" : {
                        "media_keys" : [
                                "7_1490271263876874246"
                        ]
                },
                "author_id" : "1284787885015629825",
                "conversation_id" : "1490297678731018241",
                "created_at" : "2022-02-06T12:13:49.000Z",
                "entities" : {
                        "hashtags" : [
                                {
                                        "start" : 63,
                                        "end" : 67,
                                        "tag" : "최성훈"
                                }
                        ],
                        "mentions" : [
                                {
                                        "start" : 3,
                                        "end" : 17,
                                        "username" : "dear__rabbit_",
                                        "id" : "1421338058843058180"
                                }
                        ],
                        "urls" : [
                                {
                                        "start" : 68,
                                        "end" : 91,
                                        "url" : "https://t.co/pFzGnkU6X1",
                                        "expanded_url" : "https://twitter.com/dear__rabbit_/status/1490271343166001152/video/1",
                                        "display_url" : "pic.twitter.com/pFzGnkU6X1"
                                }
                        ]
                },
                "geo" : {

                },
                "id" : "1490297678731018241",
                "lang" : "ko",
                "non_public_metrics" : {

                },
                "organic_metrics" : {

                },
                "possibly_sensitive" : false,
                "promoted_metrics" : {

                },
                "public_metrics" : {
                        "retweet_count" : 24,
                        "reply_count" : 0,
                        "like_count" : 0,
                        "quote_count" : 0
                },
                "referenced_tweets" : [
                        {
                                "type" : "retweeted",
                                "id" : "1490271343166001152"
                        }
                ],
                "reply_settings" : "everyone",
                "source" : "Twitter for iPhone",
                "text" : "RT @dear__rabbit_: 220206 이클립스콘 서울 프리뷰 \nMy reason to be brave💘 #최성훈 https://t.co/pFzGnkU6X1"
        },
        "_timestamp" : "2022-02-06T21:14:00.780454+09:00"
}

# 해시태그가 있으면서 한국어인 경우 tag만 조회: mongodb 프로젝션
db.tweet_sample.find({ $and: [ { "data.lang": "ko" }, { "data.entities.hashtags": { "$exists":true }} ] }, { "data.entities.hashtags": true })
db.tweet_sample.find({ $and: [ { "data.lang": "ko" }, { "data.entities.hashtags": { "$exists":true }} ] }, { "data.entities.hashtags": true }).pretty()
{
        "_id" : ObjectId("61ffbba8f6b8e9bd026bbcf7"),
        "data" : {
                "entities" : {
                        "hashtags" : [
                                {
                                        "start" : 17,
                                        "end" : 30,
                                        "tag" : "뽑으시오_나를_뽑으시오"
                                },
                                {
                                        "start" : 31,
                                        "end" : 49,
                                        "tag" : "내가_재밌는_폴라_하나_보여주지"
                                },
                                {
                                        "start" : 50,
                                        "end" : 72,
                                        "tag" : "여보게_자네_내가올때_짱폴라나_찍어주소"
                                }
                        ]
                }
        }
}

# 태그만 출력
db.tweet_sample.find({ $and: [ { "data.lang": "ko" }, { "data.entities.hashtags": { "$exists":true }} ] }, { "data.entities.hashtags.tag": true })
{
        "_id" : ObjectId("61ffbba3f6b8e9bd026bbc20"),
        "data" : {
                "entities" : {
                        "hashtags" : [
                                {
                                        "tag" : "한글"
                                },
                                {
                                        "tag" : "워들"
                                },
                                {
                                        "tag" : "Korean"
                                },
                                {
                                        "tag" : "Wordle"
                                }
                        ]
                }
        }
}
{
        "_id" : ObjectId("61ffbba8f6b8e9bd026bbcf7"),
        "data" : {
                "entities" : {
                        "hashtags" : [
                                {
                                        "tag" : "뽑으시오_나를_뽑으시오"
                                },
                                {
                                        "tag" : "내가_재밌는_폴라_하나_보여주지"
                                },
                                {
                                        "tag" : "여보게_자네_내가올때_짱폴라나_찍어주소"
                                }
                        ]
                }
        }
}
```